# This python script pulls price history for 76 Vanguard ETFs from 2018 to 2022

### ETFs were manually copied from Vanguard's website https://investor.vanguard.com/etf/list#/etf/asset-class/month-end-returns and placed into vanguard_etfs.csv

### This script is used with m_portfolio_allocation to generate a markowitz portfolio allocation analysis on Vanguard's ETFs

### Link to the Tableau Dashboard: https://public.tableau.com/views/All-ETFPortfolioOptimizer/All-ETFPortfolioOptimizer?:language=en-US&:display_count=n&:origin=viz_share_link

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
etfs = pd.read_csv('vanguard_etfs.csv')

### Below is the csv that was manually made from Vanguard's website

In [3]:
etfs

,Ticker,Asset Class,Expense Ratio,Price,YTD,1-year annual returns,5-year annual returns,10-year annual returns,total annual returns,ETF Start Date
0,EDV,Bond - Long-term Government,0.0006,105.93,-0.2395,-0.0077,0.0522,0.0550,0.0687,12/6/2007
1,BIV,Bond - Inter-term Investment,0.0004,78.52,-0.0991,-0.0462,0.0252,0.0273,0.0439,4/3/2007
2,VGIT,Bond - Inter-term Government,0.0004,61.28,-0.0754,-0.0495,0.0152,0.0159,0.0239,11/19/2009
3,BLV,Bond - Long-term Investment,0.0004,82.71,-0.1890,-0.0338,0.0453,0.0470,0.0614,4/3/2007
4,VGLT,Bond - Long-term Government,0.0004,72.88,-0.1809,-0.0135,0.0387,0.0394,0.0525,11/19/2009
...,...,...,...,...,...,...,...,...,...,...
71,VIS,Stock - Sector,0.0010,179.22,-0.1141,0.0406,0.1120,0.1273,0.0996,9/23/2004
72,VGT,Stock - Sector,0.0010,367.38,-0.1966,0.1697,0.2644,0.2020,0.1327,1/26/2004
73,VAW,Stock - Sector,0.0010,186.16,-0.0512,0.1402,0.1231,0.1108,0.0989,1/26/2004
74,VNQ,Stock - Sector,0.0012,103.87,-0.0992,0.2142,0.0967,0.0969,0.0926,9/23/2004


In [39]:
tickers = etfs['Ticker']

In [40]:
def get_price_data(ticker_list):
    list_of_dfs = []
    for i in ticker_list:
        ticker = i

        tickerdata = yf.Ticker(ticker)

        pricedf = tickerdata.history(period='1d',start = '2018-1-1', end = '2022-4-30')

        pricedf = pricedf.drop(columns = ['Dividends','Stock Splits'])
        
        pricedf['Ticker'] = i
        
        pricedf['Date'] = pricedf.index
                                     
        list_of_dfs.append(pricedf)
    return list_of_dfs

In [41]:
etf_data_list = get_price_data(tickers)

In [42]:
all_etfs = pd.concat(etf_data_list)

### Below is the output from using the tickers and pulling data from yahoo finance from 1/1/2018 to 4/30/2022

In [43]:
all_etfs

,Open,High,Low,Close,Volume,Ticker,Date
Date,,,,,,,
2018-01-02,104.346624,104.346624,102.709849,103.238121,86200,EDV,2018-01-02
2018-01-03,103.619171,103.861655,103.238120,103.688454,38800,EDV,2018-01-03
2018-01-04,103.471955,103.913620,103.056263,103.870323,33200,EDV,2018-01-04
2018-01-05,103.489255,103.792360,102.952321,103.385330,45500,EDV,2018-01-05
2018-01-08,103.679785,103.714427,102.874388,103.168831,48200,EDV,2018-01-08
...,...,...,...,...,...,...,...
2022-04-25,161.600006,161.779999,157.399994,159.949997,359000,VPU,2022-04-25
2022-04-26,159.360001,161.250000,158.419998,158.449997,208400,VPU,2022-04-26
2022-04-27,158.729996,159.889999,157.289993,157.589996,732300,VPU,2022-04-27


In [44]:
all_etfs.to_csv('etf_price_data.csv',index = False)

### Ticker information is also pulled from yahoo finance and placed in a csv

In [69]:
def get_ticker_info(ticker_list):
    
    list_of_etf_info = []
    for i in ticker_list:
        
        ticker = i

        tickerdata = yf.Ticker(ticker)

        #Get basic ticker info
        tickerinfo = tickerdata.info

        infocols = list(tickerinfo.keys())

        infodf = pd.DataFrame.from_dict(tickerinfo,orient='index')
        
        infodf['Ticker'] = i

        infodf.reset_index(inplace = True)

        infodf['measure'] = infodf['index']
        infodf['value'] = infodf[0]
        infodf = infodf.drop(columns = ['index',0])

        list_of_etf_info.append(infodf)
        
    return list_of_etf_info

In [65]:
etf_info_list = get_ticker_info(tickers)

In [66]:
etf_info_list[0]

,Ticker,measure,value
0,EDV,exchange,PCX
1,EDV,shortName,Vanguard Extended Duration Trea
2,EDV,longName,Vanguard Extended Duration Treasury Index Fund
3,EDV,exchangeTimezoneName,America/New_York
4,EDV,exchangeTimezoneShortName,EDT
...,...,...,...
98,EDV,dividendYield,None
99,EDV,bidSize,900
100,EDV,dayHigh,107.9699
101,EDV,regularMarketPrice,105.84


In [67]:
etf_info_df = pd.concat(etf_info_list)

In [68]:
etf_info_df.to_csv('etf_info.csv',index = False)